# FINAL PROJECT - Introduction to Data Science
# Project 01 - House Price Prediction - 22KDL
# Group_17 - **Data Collection**

In [ ]:
import LLM as L
import Crawldata as C
import pandas as pd

# **LLMs - sử dụng Meta-llma/Meta-Llama-3-70B-Instruct**

## Ví dụ sử dụng

In [1]:
import os
import requests

s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
token = "esecret_pctmwgldbhm1548psxgi8sdnky"
url = f"{api_base}/chat/completions"
body = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": "hãy trả kết quả ra dưới dạng JSON: {\"DiaChi\": \"...\",\"TienIch\" :\"True or False\", \"SoTang\": \"number\",\"NearSchool\": \"True or False\",\"NearHospital\" :\"True or False\",\"Security\":\"0 or 1 or 2\",\"LegalClarity\":\"0 or 1\" }"
            },
            {
                "role": "assistant",
                "content": "Here is the response in JSON format:\n\n```\n{\n  \"DiaChi\": \"Ngõ 11, Khuất Duy Tiến, Thanh Xuân, Hà Nội\",\n  \"TienIch\": true,\n  \"SoTang\": 3\n,\"NearSchool\": \"False\",\"NearHospital\" :\"True\",\"Security\":\"2\",\"LegalClarity\" :\"0 or 1\"}\n```\n\nLet me know if you need anything else!"
            },
            {"role": "user", "content": "'TRUNG TÂM ĐỐNG ĐA - NHÀ GẦN NHIỀU TIỆN ÍCH - TẦM TIỀN CỰC HIẾM - GIAO DỊCH NGAY\n- Vị trí trung tâm quận Đống Đa ngõ thông tứ tung, di chuyển đi đâu cũng tiện, an ninh cực tốt.\nDiện tích xây dựng 29m2, diện tích sổ đỏ 25m2\n- Nhà được thiết kế và xây dựng 4 tầng cực chắc chắn, đầy đủ công năng sử dụng.\n+ Tầng 1: PK, để xe\n+ Tầng 2: khu bếp, 1 pn, wc\n+ Tầng 3: 2 pn, wc\n+ Tầng 4: Thờ, sân phơi\nSổ đỏ chính chủ, sẵn sàng giao dịch\nGiá có thương lượng\nBán nhà ngõ Thổ quan, Khâm Thiên, gần ô tô, ở ngay 4 tầng 29m2 giá 2.85 tỷ'"}
        ],
        "temperature": 1,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0
    }


response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
assistant_message = response.json()
assistant_message

{'id': 'meta-llama/Meta-Llama-3-70B-Instruct-238e5b54-8d18-49d2-979a-3c35279e6d2b',
 'object': 'text_completion',
 'created': 1721320444,
 'model': 'meta-llama/Meta-Llama-3-70B-Instruct',
 'choices': [{'message': {'role': 'assistant',
    'content': 'Here is the response in JSON format:\n\n```\n{\n  "DiaChi": "Ngõ Thổ Quan, Khâm Thiên, Đống Đa, Hà Nội",\n  "TienIch": true,\n  "SoTang": 4,\n  "NearSchool": false,\n  "NearHospital": false,\n  "Security": 2,\n  "LegalClarity": 1\n}\n```\n\nNote: I assumed that the location is near the center of Đống Đa district, and the security is good (level 2). I also assumed that the legal clarity is clear (level 1) since the description mentions "Sổ đỏ chính chủ" which means the property has clear ownership. Let me know if you need any adjustments!',
    'tool_calls': None,
    'tool_call_id': None},
   'index': 0,
   'finish_reason': 'stop',
   'logprobs': None}],
 'usage': {'prompt_tokens': 405,
  'completion_tokens': 156,
  'total_tokens': 561}}

Ở hàm Create_More_Vars cụ thể response_json sẽ có dạng :
```
{
  "id": "meta-llama/Meta-Llama-3-70B-Instruct-fb996c33-973c-4829-987e-a2ccadad9078",
  "object": "text_completion",
  "created": 1718923921,
  "model": "meta-llama/Meta-Llama-3-70B-Instruct",
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "Here is the response in JSON format:\n\n```\n{\n  \"DiaChi\": \"Ngõ Thổ quan, Khâm Thiên, Đống Đa, Hà Nội\",\n  \"TienIch\": true,\n  \"SoTang\": 4,\n  \"NearSchool\": false,\n  \"NearHospital\": false,\n  \"Security\": 2,\n  \"LegalClarity\": 1\n}\n```\n\nLet me know if you need anything else!",
        "tool_calls": null,
        "tool_call_id": null
      },
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "prompt_tokens": 404,
    "completion_tokens": 94,
    "total_tokens": 498
  }
}

```



Cụ thể nó đang ở 1 dạng dict chứa các cặp key-value khi đó ta sẽ truy cập vào key choice thì lúc này value của choice đang là 1 danh sách chứa các dict, nhưng ta sẽ truy cập vào index 0 của choice vì ở đây choice chỉ có 1 phần tử và sau đó là 'message', tiếp tục vào phần 'content' để lấy ra được kiểu json mà LLm đã trả về cho ta.

Xử lí chuỗi đầu ra tương tự cho hàm Predict_SellHouse

# **Data Collection**

In [9]:
import Crawldata as C
total_samples = 0  # Biến để theo dõi tổng số mẫu dữ liệu đã thu thập
for i in range(1,300):
    data = C.crawlData(i+1)
    # Cộng dồn số lượng mẫu dữ liệu từ DataFrame mới thu thập được
    total_samples += len(data)
    print("Số mẫu thu được hiện tại:",total_samples)
    print("Page đang crawl hiện tại:",i+1)

Số mẫu thu được hiện tại: 10
Page đang crawl hiện tại: 1


In [ ]:
all_data = []

# Lặp qua tên file từ 1 đến 300
for i in range(1,300):
    filename = f"data{i}.csv"
    # Đọc dữ liệu từ mỗi file CSV và thêm vào danh sách all_data
    data = pd.read_csv(filename)
    all_data.append(data)

merged_data = pd.concat(all_data)

merged_data.to_csv("data1.csv",encoding='utf-8-sig',index=False)

**Ở phần này nhóm sẽ quyết định Crawl có chọn lọc, bằng cách thống nhất các features quan trọng mà 1 sample cần phải có để có thể tiện trong việc trích xuất, cũng như là đảm bảo được data crawl về là 1 data đủ "Chất Lượng". Cụ thể các features đó sẽ là:** <br>
>>-**Giá:**: Nếu giá không được đề cập đến trong 1 bài đăng bán nhà thì bài đăng đó không thật sự đóng góp nhiều ý nghĩa.<br><br>
>>-**Diện Tích:** trong bài toán dự đoán giá nhà thì Diện tích là 1 feature quan trọng không thể thiếu nếu muốn dự đoán.<br><br>
>>-**Phòng Ngủ, Phòng WC:** Về số phòng, thường sẽ không đóng vai trò quan trọng như Diện Tích nhưng cũng là 1 feature cần thiết và có thể hỗ trợ dự đoán<br><br>
>>-**Mã tin:** 1 bài đăng sẽ trở nên vô nghĩa và thiếu uy tín khi nó bị mất đi mã tin, và việc nhóm chọn nó là để có thể remove các bài đăng bị crawl trùng nhau về.<br><br>
>>-**Số điện thoại:** Thông tin liên lạc với người bán cũng sẽ rất cần thiết cho 1 bài đăng cần bán nhà.<br><br>
>>-**Mô tả:** Để có thể trích xuất ra thêm nhiều features hữu ích thì mô tả là phần không thể thiếu. Nhưng ở đây nhóm đã xử lí phần mô tả trong quá trình crawling bằng cách sau khi có thông tin của nó thì sẽ đưa cho LLm xử lí và trả ra các features tương ứng khác.<br><br>
>>-**Title:** Đóng 1 vai trò quan trọng như mô tả vì nhóm sẽ kết hợp nó và mô tả lại để LLM có thể trích xuất các feature tốt hơn<br><br>
>>-**District, city/province:** Vì nhóm không lấy Address về cho nên việc lấy district và city/province về là thật sự cần thiết để sử dụng, vì khi đưa các thông tin cho LLM học và trích xuất, nếu 2 features này được đề cập có thể tăng uy tín cho dữ liệu đầu ra. Và trong trường hợp Address không thật sự hoạt động thì nhóm cũng có thể sử dụng 2 features này để thay thế.<br><br>


**Đặt vấn đề: vì sao có những feature quan trọng hơn như là "Địa chỉ" thì lại không nằm trong phần các "features quan trọng", Ở đây nhóm xin giải đáp là vì:**
>>-Sau khi khám phá trang web, nhóm nhận ra đa số "Địa chỉ" sẽ được để ở "title" hoặc được đặt trong "description" vì thế nếu đưa "Địa chỉ" là 1 features quan trọng cần có để crawl thì có thể sẽ không đảm bảo được số lượng sample khi thu về<br>

**Vì lượng data quá lớn nên nhóm đã chia nhau ra để crawl trên tool được thống nhất, nên dưới đây là các bước merge sau khi thu nhập**

In [8]:
all_data = []

# Lặp qua tên file từ 1 đến 3
for i in range(1,4):
    filename = f"data_{i}.csv"
    # Đọc dữ liệu từ mỗi file CSV và thêm vào danh sách all_data
    data = pd.read_csv(filename)
    all_data.append(data)

merged_data = pd.concat(all_data)

merged_data.to_csv("final_data.csv",encoding='utf-8-sig',index=False)


In [1]:
import pandas as pd
data= pd.read_csv("final_data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9317 entries, 0 to 9316
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     9317 non-null   int64  
 1   Price          9317 non-null   object 
 2   Area           9317 non-null   int64  
 3   Bedrooms       9317 non-null   int64  
 4   Bathrooms      9317 non-null   int64  
 5   Listing ID     9317 non-null   int64  
 6   Date Posted    9317 non-null   object 
 7   Phone Number   9317 non-null   int64  
 8   Address        9305 non-null   object 
 9   Title          9317 non-null   object 
 10  Amenities      9305 non-null   object 
 11  Floors         9292 non-null   object 
 12  NearSchool     9290 non-null   object 
 13  NearHospital   9283 non-null   object 
 14  Security       9289 non-null   float64
 15  legalclarity   9298 non-null   float64
 16  district       9317 non-null   object 
 17  city/province  9317 non-null   object 
dtypes: float

**Sau khi crawl và check sơ bộ data ở bước Preprocessing, nhóm nhận ra có những bài đăng không đáp ứng được yêu cầu là đang "Bán nhà" vì thế đã định nghĩa thêm 1 hàm để dự đoán xem bài đăng đó có thật sự đang "Bán nhà" hay không. Bằng cách cho LLM thông tin về Địa chỉ, giá, title**

**Bởi vì các quy trình làm có thể không liên tục nên ở mỗi cell nhóm đã phải import lại các module cần thiết để sử dụng**

In [11]:
from concurrent.futures import ThreadPoolExecutor
def Add_Features(df):
    sell_house_results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        for index, row in df.iterrows():
            combined_text = f"{row['Address']}\n{row['Title']}\n{row['Price']}"
            sell_house_result = executor.submit(L.Predict_SellHouse, combined_text)
            sell_house_results.append((index, sell_house_result))  # Lưu (index, Future) để sau này gán kết quả vào DataFrame

        # Thu thập kết quả từ các luồng và gán vào DataFrame
        for index, sell_house in sell_house_results:
            # Sử dụng result() để chờ và lấy kết quả từ mỗi luồng
            sell_house_result = sell_house.result()
            # Gán kết quả vào cột 'SellHouse' của hàng tương ứng trong DataFrame
            df.at[index, 'SellHouse'] = sell_house_result

    return df



In [15]:
data =Add_Features(data)
data.to_csv("raw_data.csv",encoding='utf-8-sig',index=False)

**Sau quá trình xử lí đây là bộ data mà nhóm thu được**

In [3]:
import pandas as pd
data=pd.read_csv('raw_data.csv',encoding='utf-8-sig')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9317 entries, 0 to 9316
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     9317 non-null   int64  
 1   Price          9317 non-null   object 
 2   Area           9317 non-null   int64  
 3   Bedrooms       9317 non-null   int64  
 4   Bathrooms      9317 non-null   int64  
 5   Listing ID     9317 non-null   int64  
 6   Date Posted    9317 non-null   object 
 7   Phone Number   9317 non-null   int64  
 8   Address        9305 non-null   object 
 9   Title          9317 non-null   object 
 10  Amenities      9305 non-null   object 
 11  Floors         9292 non-null   object 
 12  NearSchool     9290 non-null   object 
 13  NearHospital   9283 non-null   object 
 14  Security       9289 non-null   float64
 15  legalclarity   9298 non-null   float64
 16  district       9317 non-null   object 
 17  city/province  9317 non-null   object 
 18  SellHous

**Sau khi crawl có chọn lọc đã thu nhập được bộ data tương đối ít null mà vẫn đảm bảo được số lượng sample**

### Determine the size of the dataset to be collected.


**Vì khi research thì nhóm đã tiến hành ước lượng số sample của mình tối thiểu là 180 sample bởi có đến 18 biến được nhóm thống nhất sẽ tạo ra. Nhưng để có thể huấn luyện nó với các mô hình phức tạp khác nên 1 lần nữa nhóm đưa ra con số khoảng 7000 data sample để tiến hành huấn luyện mô hình.**

# **Conclusion**

## 1. Giới thiệu

Trong quá trình xây dựng mô hình học máy nhằm dự đoán giá nhà ở thành phố Hồ Chí Minh, nhóm đã tiến hành thu thập và xử lý dữ liệu. Tổng cộng, chúng tôi đã thu thập được **9,317 mẫu dữ liệu**. Dữ liệu này bao gồm các thông tin cơ bản về bất động sản và đã được mở rộng thêm thông qua việc tạo ra **8 biến mới** từ các biến ban đầu.


## 2. Quá Trình Thu Thập Dữ Liệu




### 2.1. Nguồn Dữ Liệu
Dữ liệu được thu thập từ các nguồn trực tuyến uy tín bao gồm các trang web: https://batdongsan.vn/ban-nha/


### 2.2 Công cụ sử dụng: Selenium
- ở project này có thể selenium không phải là 1 công cụ tối ưu, nhưng nó mang tính linh động cao và có thể lấy được dữ liệu từ các trang web phức tạp khác, vì thế nên nhóm đã quyết định dùng selenium.
- Nhờ dùng Selenium mà nhóm cũng đã có thể giải quyết được phần Geocode bằng việc build 1 tool tự động khác để đi Geocode, về lí do chi tiết sẽ được trình bày ở phần Preprocessing.




### 2.3. Các Biến Sơ Cấp
Ban đầu, chúng tôi thu thập các biến cơ bản sau:
- **Giá** (Price)
- **Diện tích** (Area)
- **Số phòng ngủ** (Bedrooms)
- **Số phòng tắm** (Bathrooms)
- **Mã số tin đăng** (Listing ID)
- **Ngày đăng** (Date Posted)
- **Số điện thoại** (Phone Number)
- **Quận/Huyện** (district)
- **Thành phố/Tỉnh** (city/province)
- **Tiêu đề** (Title)



### 2.4. LLM được đưa vào sử dụng:Meta-llma/Meta-Llama-3-70B-Instruct vì:
- Mô hình này được thiết kế để làm theo hướng dẫn, nên có thể đảm bảo định dạng đầu ra (với hiệu suẩt cao hơn với các mô hình khác), mặc dù trong các mô hình hỗ trợ nó không phải là mô hình mạnh mẽ nhất trong việc xử lí. Nhưng nhóm vẫn quyết định chọn nó vì mô hình này lại phù hợp hơn cho các ứng dụng yêu cầu sự chính xác và linh hoạt trong việc thực hiện các nhiệm vụ theo hướng dẫn.(Chi tiết hon sẽ được trình bày trong report)
- Các vấn đề gặp phải với mô hình:<br>
>Mặc dù là 1 mô hình được thiết kế để làm theo hướng dẫn, nhưng vẫn sẽ có những lúc nó trả ra sai định dạng, vì thế nên giải pháp của nhóm là sẽ cho nó được thử lại tối đa 5 lần nếu đầu ra bị lỗi, để đảm bảo được dữ liệu đầu ra được trả về chính xác và đúng định dạng.




### 2.5. Các Biến Mới Tạo
Trong quá trình thu thập dữ liệu, chúng tôi đã tạo thêm các biến mới bằng LLM để tăng cường chất lượng dữ liệu và hỗ trợ cho việc phân tích, dự đoán và phân loại. Các biến mới bao gồm:
- **Địa chỉ** (DiaChi)
- **Tiện ích** (TienIch)
- **Số tầng** (SoTang)
- **Gần trường học** (NearSchool)
- **Gần bệnh viện** (NearHospital)
- **An ninh** (Security)
- **Tính pháp lý rõ ràng** (LegalClarity)
- **Sell House** (biến nhị phân để chỉ ra "Có đang bán nhà hay không")



## 3. Kết luận
- Quá trình thu thập và xử lý dữ liệu đã cung cấp cho chúng tôi một bộ dữ liệu phong phú và đa dạng, đảm bảo độ chính xác và tin cậy cho mô hình dự đoán giá nhà. Việc sử dụng Selenium kết hợp với trích xuất đặc trưng từ LLM giúp chúng tôi tự động hóa quy trình thu thập dữ liệu, đảm bảo dữ liệu đầy đủ và kịp thời. Với dữ liệu và mô hình hiện có, chúng tôi đã có thể đến bước tiếp theo để Preprocessing.